# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — мы её уже сделали.

Построим модель с максимально большим значением *accuracy*. Проект можно считать успешным, если довести долю правильных ответов по крайней мере до 0.75. Проверять *accuracy* будем на тестовой выборке.

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.dummy import DummyClassifier

import warnings
warnings.filterwarnings('ignore')

## Знакомство с данными


In [ ]:
df = pd.read_csv('/datasets/users_behavior.csv')

display(df.head())
display(df.info())
display(df.describe())

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


None

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


В наборе данных представлена информация о количестве звонков, потраченных минут, отправленных сообщений и количестве использованного интернет-трафика (в МБ), а также то, перешёл ли пользователь на тариф 'Ультра' или, в противном случае, на тариф 'Смарт'

Пропущенных действительно данных нет, т.к. данные уже прошли предобработку в одном из предыдущих проектов

Статистические данные показывают, что распределение соответствует рельно возможному, а выбросы отсутствуют.

## Разбивка на выборки

Разбили данные на выборки и получили 3 выборки: обучающую, валидационную и тестовую. Предварительно выделили признаки и целевой признак:

In [ ]:
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

train_features, test_features, train_target, test_target = train_test_split(features, target,
                                                                test_size=0.4, random_state=52, stratify=target)
test_features, valid_features, test_target, valid_target = train_test_split(test_features, test_target,
                                                                test_size=0.5, random_state=52, stratify=test_target)

print(train_features.shape, test_features.shape, valid_features.shape)

(1928, 4) (643, 4) (643, 4)


## Исследование моделей

Так как перед нами стоит задача классификации, мы применяли в дальнешей работе соответствующие модели - модели классификации. Подбирали гиперпараметры для модели в поисках наиболее подходящего варианта вручную.


### Классификация Дерева решений



In [ ]:
best_tree_result = 0
for depth in range(1,11):
    tree_model = DecisionTreeClassifier(random_state=52, max_depth=depth)
    tree_model.fit(train_features, train_target)
    tree_result = tree_model.score(valid_features, valid_target)
    if tree_result > best_tree_result:
        best_tree_result = tree_result
        best_depth = depth
print('Лучший результат:',best_tree_result,'Наиболее успешная глубина:',best_depth)

Лучший результат: 0.7807153965785381 Наиболее успешная глубина: 3


### Классификация Случайного леса


In [ ]:
best_forest_result = 0
for est in range(10, 36, 5):
    for depth in range (1, 11):
        forest_model = RandomForestClassifier(random_state=52, n_estimators=est, max_depth=depth)
        forest_model.fit(train_features, train_target)
        forest_result = forest_model.score(valid_features, valid_target)
        if forest_result > best_forest_result:
            best_forest_result = forest_result
            best_depth = depth
            best_est = est
print('Лучший результат:',best_forest_result,'Наиболее успешное количество деревьев:',best_est,\
      'Наиболее успешная глубина:',best_depth)

Лучший результат: 0.8087091757387247 Наиболее успешное количество деревьев: 10 Наиболее успешная глубина: 5


### Классификация Логистической регрессии

In [ ]:
regr_model = LogisticRegression(random_state=52)
regr_model.fit(train_features, train_target)
regr_model_result = regr_model.score(valid_features, valid_target)
print('Результат:',regr_model_result)

Результат: 0.7480559875583204


### Наивная Бейсовская классификация

In [ ]:
Gaus_model = GaussianNB()
Gaus_model.fit(train_features, train_target)
Gaus_result = Gaus_model.score(valid_features, valid_target)
print('Результат :',Gaus_result)

Результат : 0.7791601866251944


### Вывод по успешности моделей

In [ ]:
print(f'Показатели accuracy для разных моделей:\n\n\
      Дерево решений:\t\t{best_tree_result}\n\
      Случайный лес:\t\t{best_forest_result}\n\
      Логистическая регрессия:\t{regr_model_result}\n\
      Бейсовская классификация:\t{Gaus_result}')

Показатели accuracy для разных моделей:

      Дерево решений:		0.7807153965785381
      Случайный лес:		0.8087091757387247
      Логистическая регрессия:	0.7480559875583204
      Бейсовская классификация:	0.7791601866251944


Наиболее подходящими гиперпараметрами для дерева решений оказалась глубина дерева равная 8.
А для случайного леса количество деревьев равное 35 и глубина равная 10.

Наивысшую точность показала модель случайного леса, её мы и проверим на тестовой выборке.

## Проверка модели на тестовой выборке

Воссоздали наиболее точную модель с подходящими гиперпараметрами и рассчитали точность предсказания на тестовой выборке:

In [ ]:
model = RandomForestClassifier(random_state=52, n_estimators=best_est, max_depth=best_depth)
model.fit(train_features, train_target)
result = model.score(test_features, test_target)
print('Результат проверки модели случайного леса на тестовой выборке:',result)

Результат проверки модели случайного леса на тестовой выборке: 0.8258164852255054


## Проверка модели на адекватность

Создали случайную модель и рассчитали её точность:


In [ ]:
dum_model = DummyClassifier(strategy='most_frequent', random_state=52)
dum_model.fit(train_features, train_target)
result = dum_model.score(test_features, test_target)
print('Результат проверки случайной модели на тестовой выборке:',result)

Результат проверки случайной модели на тестовой выборке: 0.6936236391912908


## Итоговый вывод

1. Данные для построения моделей были загружены и изучены, успешная преобработка была подтверждена.
2. Данные были разбиты на обучающую, валидационную и тестовую выборки.
3. Были построены различные модели для задачи классификации (Дерево решений, Случайный лес, Логистическая регрессия, Гауссовы процессы), а также были подобраны наиболее подходящие гиперпараметры.
    - Наиболее успешная глубина дерева решений - 8
    - Наиболее успешное количество деревьев - 35, наиболее успешная глубина дерева - 10 для случайного леса.
    - Наиболее успешной моделью оказался **Случайный лес**.
4. Проведено тестирование наиболее успешной модели (Случайный лес) на тестовой выборке и получено крайне малое расхождение с точностью на валидационной выборке
    - Точность выбранной модели на **валидационной выборке - 0.807153965785381**
    - Точность выбранной модели на **тестовой выборке - 0.7993779160186625**
5. Проведена проверека на вменяемость на случаной модели и получено существенное расхождение точности, что говорит о вменяемости модели
    - Точность **случайной модели - 0.6656298600311042**
    - Точность **выбранной модели - 0.807153965785381**